In [2]:
directory = "/data/s1968653/Vanilla_Tack_output/"
run_number = 8

In [ ]:
#Here we import all the necessary dependencies
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from IPython.display import clear_output
from amuse.lab import units, constants
from amuse.ext.orbital_elements import new_binary_from_orbital_elements
from amuse.ext.orbital_elements import get_orbital_elements_from_binary
from amuse.ext.orbital_elements import get_orbital_elements_from_binaries
from amuse.lab import Particles
from amuse.lab import nbody_system
from amuse.couple import bridge
from amuse.lab import Rebound
from amuse.community.mercury.interface import MercuryWayWard
from amuse.lab import Mercury
from amuse.community.ph4.interface import ph4
from amuse.io import write_set_to_file, read_set_from_file
from amuse.ext.orbital_elements import get_orbital_elements_from_binary
import amuse.plot as plot
from scipy.optimize import curve_fit
from amuse.lab import Huayno
import math

In [ ]:
def sma_determinator(primary, secondary):
    binary = Particles(0)
    binary.add_particle(primary)
    binary.add_particle(secondary)
        
    orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
    return orbital_params[2]

In [ ]:
# Function to generate random orbits for asteroids in the Solar System.
def random_positions_and_velocities(N_objects, sun_loc):  
    positions = np.zeros((N_objects, 3)) | units.AU
    velocities = np.zeros((N_objects,3)) | units.kms
    
    m_sun = 1 | units.MSun
    m_oort = 0 | units.MSun
    for i in range(N_objects):
        # Values below correspond with random locations anywhere in the Solar System
        a = np.random.uniform(4, 40) | units.AU  # semi-major axis
        e = np.random.uniform(0, 0.05)  # eccentricity
        inclination = np.random.uniform(-5, 5) | units.deg
        true_anomaly = np.random.uniform (0, 360) | units.deg
        arg_of_periapsis = np.random.uniform(0, 360) | units.deg
        long_of_ascending_node = np.random.uniform(0, 360) | units.deg
        sun_and_oort = new_binary_from_orbital_elements(m_sun, m_oort, 
                                          a, e, true_anomaly, inclination, long_of_ascending_node, arg_of_periapsis, G=constants.G)
        positions[i] = (sun_and_oort[1].x+sun_loc[0]), (sun_and_oort[1].y+sun_loc[1]), (sun_and_oort[1].z+sun_loc[2])
        velocities[i]= sun_and_oort[1].vx, sun_and_oort[1].vy, sun_and_oort[1].vz
    return positions, velocities

In [ ]:
def create_tack():
    sun = Particles(1)
    sun.name = "Sun"
    sun.mass = 1.0 | units.MSun
    sun.radius = 1.0 | units.RSun  
    sun.position = (0, 0, 0) | units.AU
    sun.velocity = (0, 0, 0) | units.kms
    sun.density = 3*sun.mass/(4*np.pi*sun.radius**3)
    
    names = ["Jupiter", "Saturn", "Uranus", "Neptune"]
    masses = [317.8, 30, 5, 5] | units.MEarth
    radii = [0.10049, 0.083703, 0.036455, 0.035392] | units.RSun
    semis = [3.5, 4.5, 6.013, 8.031] | units.AU #[3.5, 4.9, 6.4, 8.4]
    trues = [0, 90, 180, 270]| units.deg#np.random.uniform(0, 360, 4) | units.deg
    longs = np.random.uniform(0, 360, 4) | units.deg
    args = np.random.uniform(0, 360, 4) | units.deg
    
    for i in range(4):
        sun_and_plan = new_binary_from_orbital_elements(sun[0].mass, masses[i], 
                                          semis[i], 0, trues[i], np.random.uniform(-5, 5) | units.deg , longs[i], args[i], G=constants.G)
        
        planet = Particles(1)
        planet.name = names[i]
        planet.mass = masses[i]
        planet.radius = radii[i] # This is purely non-zero for collisional purposes
        planet.position = (sun_and_plan[1].x-(0 | units.AU), sun_and_plan[1].y-(0 | units.AU), sun_and_plan[1].z-(0 | units.AU))
        planet.velocity = (sun_and_plan[1].vx-(0 | units.kms), sun_and_plan[1].vy-(0 | units.kms), sun_and_plan[1].vz-(0 | units.kms))
        planet.density = 3*masses[i]/(4*np.pi*planet.radius**3)
        sun.add_particle(planet)
        
    return sun
        
basic_giants_system = create_tack()
basic_giants_system.move_to_center()

In [ ]:
#Define the number of asteroids and create random velocities and positions
N_objects = 1*10**2
sun_loc = [basic_giants_system[0].x.in_(units.AU), basic_giants_system[0].y.in_(units.AU), basic_giants_system[0].z.in_(units.AU)]
positions, velocities = random_positions_and_velocities(N_objects, sun_loc)

In [ ]:
# Here we add the asteroids, where orbit parameters were chosen from a uniform distribution
def add_comet_objects(system, N_objects, rand_pos, rand_vel):
    for i in tqdm(range(N_objects)):
        oort = Particles(1)
        oort.name = "OORT_" + str(i)
        oort.mass = 0.0 | units.MSun
        oort.radius = (2.3 | units.km).in_(units.RSun) # This is purely non-zero for collisional purposes
        oort.position = (rand_pos[i, 0], rand_pos[i, 1], rand_pos[i, 2])
        oort.velocity = (rand_vel[i, 0], rand_vel[i, 1], rand_vel[i, 2])
        oort.density = 0.0 | (units.MSun/(units.RSun**3))

        system.add_particle(oort)
    return system

complete_system = add_comet_objects(basic_giants_system, N_objects, positions, velocities)

In [ ]:
#Here we create the conditions for the migration of the planets
def semi_major_axis_next_step_in(time_now, time_scale, a_start, a_end):
    travel_distance = a_start-a_end
    sma_next_step = a_start - travel_distance*(1/(1-1/math.e))*(1-np.exp(-(time_now)/time_scale))
    return sma_next_step
    
def semi_major_axis_next_step_in_sat(time_now, time_scale, a_start, a_end):
    travel_distance = a_start-a_end
    sma_next_step = a_start - travel_distance*(1/(1-1/math.e))*(1-np.exp(-(time_now-(10**5 | units.yr))/time_scale))
    return sma_next_step

def semi_major_axis_next_step_out(time_now, time_start, a_end, a_start, time_scale):    
    travel_distance = a_end-a_start
    sma_next_step = a_start + travel_distance*(1/(1-1/math.e))*(1-np.exp(-(time_now-time_start)/time_scale))
    return sma_next_step

In [ ]:
#Here we perform the conversion for the system
final_system = complete_system
converter_length = get_orbital_elements_from_binary(final_system[0:2], G = constants.G)[2].in_(units.AU) # Typical distance used for calculation (=distance from Sun to Jupiter)
final_converter=nbody_system.nbody_to_si(final_system.mass.sum(), 
                                   converter_length)

In [ ]:
jupiter_inward_times = np.arange(0, 1*10**5, 3) 
saturn_inward_times = np.arange(1*10**5, 1.025*10**5, 0.5) 
outward_times = np.arange(1.025*10**5, 6*10**5, 15) 
post_tack_times = np.arange(6*10**5, 10**8, 5*10**3) 

final_time_range = np.concatenate((jupiter_inward_times, saturn_inward_times, outward_times, post_tack_times)) | units.yr
save_file_times = np.concatenate((jupiter_inward_times[::int(len(jupiter_inward_times)/5)], 
                                  saturn_inward_times[::int(len(saturn_inward_times)/5)], 
                                  outward_times[::int(len(outward_times)/5)], 
                                  post_tack_times[::int(len(post_tack_times)/20)])) | units.yr

In [ ]:
def vanilla_evolver(particle_system, converter, N_objects, times, save_file_times):
    
    names = ['Jupiter', 'Saturn', 'Uranus', 'Neptune']
    
    gravity_code = MercuryWayWard()
    gravity_code.initialize_code()
    
    gravity_code.central_particle.add_particle(particle_system[0])
    gravity_code.orbiters.add_particles(particle_system[1:])
    gravity_code.commit_particles
    
    ch_g2l = gravity_code.particles.new_channel_to(particle_system)
    
    #----------------------------------------------------------------------------------------------------
    
    initial_sma = [3.5, 4.5, 6.013, 8.031] | units.AU
    saturn_sma = [1.5, 0, 6.013, 8.031] | units.AU
    outward_sma = [1.5, 1.5*((3/2)**(2/3)), 6.013, 8.031] | units.AU
    sma = [3.5, 4.5, 6.013, 8.031] | units.AU
    #----------------------------------------------------------------------------------------------------
    inclinations = [0, 0, 0, 0] | units.deg
    
    for k in range(4):
        orbital_elements = get_orbital_elements_from_binary(particle_system[0]+ particle_system[k+1], G=constants.G)
        inclinations[k] =  orbital_elements[5]
    #----------------------------------------------------------------------------------------------------
    
    a_start = [1.5, 1.5*((3/2)**(2/3)), 0, 0] | units.AU
    a_end = [5.4, 5.4*((3/2)**(2/3)), 5.4*((3/2)**(2/3)*(9/5)**(2/3)), 5.4*((3/2)**(2/3)*(5/2)**(2/3))] | units.AU
    time_start = [1.025*10**5, 1.025*10**5, 0, 0] | units.yr
    time_scale = [5*10**5, 5*10**5, 0, 0] | units.yr
    
    resonances = [2/3, 3/2, 9/5, 5/2]
    pre_resonant = [False, False, True, True]
    
    saturn_migration_started = False
    outward_migration_started = False
    
    dead_comets = []
    
    
    
    #----------------------------------------------------------------------------------------------------
    
    for i in tqdm(range(len(times)-1)):
        gravity_code.evolve_model(times[i])
        ch_g2l.copy()
        
        if times[i] in save_file_times:
            write_set_to_file(gravity_code.orbiters, directory + 'Vanilla_Tack_run' + str(run_number)+ '_time=' + str(np.log10(times[i].value_in(units.yr)))[0:5] + '.hdf5', format='hdf5', overwrite_file = True)
        
        
        for j in range(4):
            sma[j] = sma_determinator(gravity_code.central_particle, gravity_code.orbiters[j])
        #----------------------------------------------------------------------------------------------------
            
        posjup = gravity_code.particles[1].position.length()
        possat = gravity_code.particles[2].position.length()
        posura = gravity_code.particles[3].position.length()
        posnep = gravity_code.particles[4].position.length()
        
        #-----------------------------------------------------------------------------------------------------
        
        if times[i] < 10**5 | units.yr :
            sma_next_step = semi_major_axis_next_step_in(times[i+1], 10**5 | units.yr, 3.5 | units.AU, 1.5 | units.AU)
            
            binary = Particles(0)
            binary.add_particle(gravity_code.central_particle)
            binary.add_particle(gravity_code.orbiters[0])

            orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
            true_anomaly, inclination, ascending_node, pericenter = orbital_params[4].in_(units.deg), orbital_params[5].in_(units.deg), orbital_params[6].in_(units.deg), orbital_params[7].in_(units.deg)

            sun_and_plan = new_binary_from_orbital_elements(1 | units.MSun, orbital_params[1], 
                                              sma_next_step, 0, true_anomaly, inclinations[0], ascending_node, pericenter, G=constants.G)

            gravity_code.particles[1].position = (sun_and_plan[1].x-(0 | units.AU), sun_and_plan[1].y-(0 | units.AU), sun_and_plan[1].z-(0 | units.AU))
            gravity_code.particles[1].velocity = (sun_and_plan[1].vx-(0 | units.kms), sun_and_plan[1].vy-(0 | units.kms), sun_and_plan[1].vz-(0 | units.kms))
            
            gravity_code.particles[2].mass *= 2**(1.5/(10**5))
            gravity_code.particles[2].mass *= 1.2**(1.5/(10**5))
            gravity_code.particles[2].mass *= 1.2**(1.5/(10**5))
            
            for j in range(3):
                binary = Particles(0)
                binary.add_particle(gravity_code.central_particle)
                binary.add_particle(gravity_code.orbiters[j+1])

                orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
                true_anomaly, inclination, ascending_node, pericenter = orbital_params[4].in_(units.deg), orbital_params[5].in_(units.deg), orbital_params[6].in_(units.deg), orbital_params[7].in_(units.deg)

                sun_and_plan = new_binary_from_orbital_elements(1 | units.MSun, orbital_params[1], 
                                              initial_sma[1+j], 0, true_anomaly, inclinations[j+1], ascending_node, pericenter, G=constants.G)

                gravity_code.particles[j+2].position = (sun_and_plan[1].x-(0 | units.AU), sun_and_plan[1].y-(0 | units.AU), sun_and_plan[1].z-(0 | units.AU))
                gravity_code.particles[j+2].velocity = (sun_and_plan[1].vx-(0 | units.kms), sun_and_plan[1].vy-(0 | units.kms), sun_and_plan[1].vz-(0 | units.kms))

        #------------------------------------------------------------------------------------------------------------
        elif 1*10**5 | units.yr <= times[i] < 1.025*10**5 | units.yr:
            if saturn_migration_started == False:
                for m in range(4):
                    saturn_sma[m] = sma[m]
                print("De saturn_smas zijn nu", saturn_sma)
                saturn_migration_started = True
                saturn_sma = [1.5, 0, 6.013, 8.031] | units.AU
                
            sma_next_step = semi_major_axis_next_step_in_sat(times[i+1], 2.5*10**3 | units.yr, 4.5 | units.AU, 1.5*((3/2)**(2/3)) | units.AU)
            
            binary = Particles(0)
            binary.add_particle(gravity_code.central_particle)
            binary.add_particle(gravity_code.orbiters[1])

            orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
            true_anomaly, inclination, ascending_node, pericenter = orbital_params[4].in_(units.deg), orbital_params[5].in_(units.deg), orbital_params[6].in_(units.deg), orbital_params[7].in_(units.deg)

            sun_and_plan = new_binary_from_orbital_elements(1 | units.MSun, orbital_params[1], 
                                              sma_next_step, 0, true_anomaly, inclinations[1], ascending_node, pericenter, G=constants.G)

            gravity_code.particles[2].position = (sun_and_plan[1].x-(0 | units.AU), sun_and_plan[1].y-(0 | units.AU), sun_and_plan[1].z-(0 | units.AU))
            gravity_code.particles[2].velocity = (sun_and_plan[1].vx-(0 | units.kms), sun_and_plan[1].vy-(0 | units.kms), sun_and_plan[1].vz-(0 | units.kms))
            
            gravity_code.particles[2].mass *= 1.5**(0.5/(2500))
            gravity_code.particles[3].mass *= (17.15/6)**(0.5/(5*10**5))
            gravity_code.particles[4].mass *= (14.54/6)**(0.5/(5*10**5))
            
            for j in [0, 2, 3]:
                binary = Particles(0)
                binary.add_particle(gravity_code.central_particle)
                binary.add_particle(gravity_code.orbiters[j])

                orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
                true_anomaly, inclination, ascending_node, pericenter = orbital_params[4].in_(units.deg), orbital_params[5].in_(units.deg),  orbital_params[6].in_(units.deg), orbital_params[7].in_(units.deg)

                sun_and_plan = new_binary_from_orbital_elements(1 | units.MSun, orbital_params[1], 
                                              saturn_sma[j], 0, true_anomaly, inclinations[j], ascending_node, pericenter, G=constants.G)

                gravity_code.particles[j+1].position = (sun_and_plan[1].x-(0 | units.AU), sun_and_plan[1].y-(0 | units.AU), sun_and_plan[1].z-(0 | units.AU))
                gravity_code.particles[j+1].velocity = (sun_and_plan[1].vx-(0 | units.kms), sun_and_plan[1].vy-(0 | units.kms), sun_and_plan[1].vz-(0 | units.kms))

        #------------------------------------------------------------------------------------------------------------    
        
        elif 1.025*10**5 | units.yr <= times[i] < 6*10**5 | units.yr:
            if outward_migration_started == False:
                for m in range(4):
                    outward_sma[m] = sma[m]
                outward_migration_started = True
                outward_sma = [1.5, 1.5*((3/2)**(2/3)), 6.013, 8.031] | units.AU
                
            
            for k in range(4):
                if pre_resonant[k] == True and k != 3:
                    if sma[k]/sma[k-1] < (resonances[k])**(2/3):
                        pre_resonant[k] = False
                        a_start[k] = sma[k]
                        time_start[k] = times[i]
                        time_scale[k] = (6*10**5 | units.yr)-times[i]
                elif pre_resonant[k] == True:
                    if sma[k]/sma[1] < (resonances[k])**(2/3):
                        pre_resonant[k] = False
                        a_start[k] = sma[k]
                        time_start[k] = times[i]
                        time_scale[k] = (6*10**5 | units.yr)-times[i]
                        
            for l in range(4):
                if pre_resonant[l] == False: 
                    sma_next_step = semi_major_axis_next_step_out(times[i+1], time_start[l], a_end[l], a_start[l], time_scale[l])
                    
                    binary = Particles(0)
                    binary.add_particle(gravity_code.central_particle)
                    binary.add_particle(gravity_code.orbiters[l])

                    orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
                    true_anomaly, inclination, ascending_node, pericenter = orbital_params[4].in_(units.deg), orbital_params[5].in_(units.deg), orbital_params[6].in_(units.deg), orbital_params[7].in_(units.deg)

                    sun_and_plan = new_binary_from_orbital_elements(1 | units.MSun, orbital_params[1], 
                                                      sma_next_step, 0, true_anomaly, inclinations[l], ascending_node, pericenter, G=constants.G)

                    gravity_code.particles[l+1].position = (sun_and_plan[1].x-(0 | units.AU), sun_and_plan[1].y-(0 | units.AU), sun_and_plan[1].z-(0 | units.AU))
                    gravity_code.particles[l+1].velocity = (sun_and_plan[1].vx-(0 | units.kms), sun_and_plan[1].vy-(0 | units.kms), sun_and_plan[1].vz-(0 | units.kms))
                               
                else:
                    binary = Particles(0)
                    binary.add_particle(gravity_code.central_particle)
                    binary.add_particle(gravity_code.orbiters[l])

                    orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
                    true_anomaly, inclination, ascending_node, pericenter = orbital_params[4].in_(units.deg), orbital_params[5].in_(units.deg), orbital_params[6].in_(units.deg), orbital_params[7].in_(units.deg)

                    sun_and_plan = new_binary_from_orbital_elements(1 | units.MSun, orbital_params[1], 
                                                      outward_sma[l], 0, true_anomaly, inclinations[l], ascending_node, pericenter, G=constants.G)

                    gravity_code.particles[l+1].position = (sun_and_plan[1].x-(0 | units.AU) , sun_and_plan[1].y-(0 | units.AU), sun_and_plan[1].z-(0 | units.AU))
                    gravity_code.particles[l+1].velocity = (sun_and_plan[1].vx-0, sun_and_plan[1].vy-(0 | units.kms), sun_and_plan[1].vz-(0 | units.kms))

            gravity_code.particles[3].mass *= (17.15/6)**(15/(5*10**5))
            gravity_code.particles[4].mass *= (14.54/6)**(15/(5*10**5))
            
        #------------------------------------------------------------------------------------------------------------
        
        else:
          #---------------------------------------------------------------------------------------------------------------
            correct_sma = [5.4, 7.1, 10.5, 13.] | units.AU
          
            for l in range(4):
                if abs(sma[l]/correct_sma[l]) > 1.25 or abs(sma[l]/correct_sma[l]) < 0.75:
                    return
          
                elif abs(sma[l]/correct_sma[l]) > 1.05 or abs(sma[l]/correct_sma[l]) < 0.95:
                    print("Here", names[l], "was redefined")
                    binary = Particles(0)
                    binary.add_particle(gravity_code.central_particle)
                    binary.add_particle(gravity_code.orbiters[l])
  
                    orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
                    true_anomaly, ascending_node, pericenter = orbital_params[4].in_(units.deg), orbital_params[6].in_(units.deg), orbital_params[7].in_(units.deg)
  
                    sun_and_plan = new_binary_from_orbital_elements(1 | units.MSun, orbital_params[1], 
                                                        correct_sma[l], 0, true_anomaly, inclinations[l], ascending_node, pericenter, G=constants.G)
  
                    gravity_code.particles[l+1].position = (sun_and_plan[1].x-(0 | units.AU), sun_and_plan[1].y-(0 | units.AU), sun_and_plan[1].z-(0 | units.AU))
                    gravity_code.particles[l+1].velocity = (sun_and_plan[1].vx-(0 | units.kms), sun_and_plan[1].vy-(0 | units.kms), sun_and_plan[1].vz-(0 | units.kms))
                else:
                    pass
        #----------------------------------------------------------------------------------------------------------------------
        
        
        
        
        
            out_of_bounds, escaped_comets = [], []
            for i in range(len(particle_system)):
                if particle_system[i].position.length() > 500 | units.AU:
                    escaped_comets.append(particle_system[i].name)
                if particle_system[i].position.length() > 250000 | units.AU:
                    out_of_bounds.append(particle_system[i])
                    dead_comets.append(particle_system[i])
            for particle in out_of_bounds:
                particle_system.remove_particle(particle)
                particle_system.synchronize_to(gravity_code.particles)
            if i%1000 == 0:
                print("The amount of currently escaped comets is ", len(escaped_comets))
                print("The amount of dead comets is ", len(dead_comets))
        
        if i%1000 == 0:
            print("The planetary postions are: ", posjup.in_(units.AU), possat.in_(units.AU), posura.in_(units.AU), posnep.in_(units.AU))
            print("The sma's are: ", sma[0], sma[1], sma[2], sma[3])
    
    gravity_code.stop()
    return particle_system
    

vanilla_evolved_system = vanilla_evolver(final_system, final_converter, N_objects, final_time_range, save_file_times)